In [ ]:
import os
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
%matplotlib inline 
from patchify import patchify
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import random

In [ ]:
### Configuring Nvidia GPU

In [ ]:
def get_mkl_enabled_flag():

    mkl_enabled = False
    major_version = int(tf.__version__.split(".")[0])
    minor_version = int(tf.__version__.split(".")[1])
    if major_version >= 2:
        if minor_version < 5:
            from tensorflow.python import _pywrap_util_port
        else:
            from tensorflow.python.util import _pywrap_util_port
            onednn_enabled = int(os.environ.get('TF_ENABLE_ONEDNN_OPTS', '0'))
        mkl_enabled = _pywrap_util_port.IsMklEnabled() or (onednn_enabled == 1)
    else:
        mkl_enabled = tf.pywrap_tensorflow.IsMklEnabled()
    return mkl_enabled

print ("We are using Tensorflow version", tf.__version__)
print("MKL enabled :", get_mkl_enabled_flag())

In [ ]:
### creating subimages

In [ ]:
root_directory = "U-NET_images/Both_Images/"
output_directory = "U-NET_images/Both_Images/128_patches/images/"
patch_size = 128
img_dir=root_directory+"images/"
  
images = os.listdir(img_dir)  #List of all image names in this subdirectory
#print(images)
for i, image_name in enumerate(images):  
    if image_name.endswith(".tif"):
        Image.MAX_IMAGE_PIXELS = None # to be able read images exceeding default pixil size
        #print(image_name)
        image = cv2.imread(img_dir+"/"+image_name, 1)  #Read each image as BGR
        print(image.shape)
        width = image.shape[1]-image.shape[1]%patch_size
        height = image.shape[0]-image.shape[0]%patch_size
        print(height, width)
        img = image[0:height, 0:width] # crops image to fit tiling
        print("padded image shape = ", img.shape)
        cv2.imwrite(root_directory+"image.tif", img)
           
        #Extract patches from each image
        print("patchifying image:", img_dir+"/"+image_name)
        patches_img = patchify(img, (128, 128, 3), step=128)  #Step=128 for 128 patches means no overlap
        cv2.imwrite(root_directory+"patched_image.tif",patches_img)
    
        for i in range(patches_img.shape[0]):
            for j in range(patches_img.shape[1]):
                single_patch_img = patches_img[i,j,:,:]
                single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds.
                                           
                print(single_patch_img.shape)
                cv2.imwrite(root_directory+"128_patches\images/"+ os.path.basename(image_name).split(".")[0]+"_patch_"+str(i)+"_"+str(j)+".tif", single_patch_img)                  

In [ ]:
#### Saving images with enough pixel information

In [ ]:
train_img_dir = "U-NET_images/Both_Images/128_patches/images/"
train_mask_dir = "U-NET_images/Both_Images/128_patches/masks/"

img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)
num_images = len(os.listdir(train_img_dir))
useless=0  #Useless image counter
for img in range(len(img_list)):   #Using t1_list as all lists are of same size
    img_name=img_list[img]
    mask_name = msk_list[img]
    print("Preparing image and masks number: ", img)   
    temp_image=cv2.imread(train_img_dir+img_list[img], 1)
   
    temp_mask=cv2.imread(train_mask_dir+msk_list[img], 0)
       
    val, counts = np.unique(temp_image, return_counts=True)
    
    if (1 - (counts[0]/counts.sum())) > 0.99:# saving images with 99% of images with information
    
        print("Save Me")
        cv2.imwrite("U-NET_images/Both_Images/128_patches/images_with_useful_info/images/"+img_name, temp_image)
        cv2.imwrite("U-NET_images/Both_Images/128_patches/images_with_useful_info/masks/"+mask_name, temp_mask)
        
    else:
        print("I am useless")   
        useless +=1

print("Sum of useful images: ", len(img_list)-useless)  
print("Sum of useless images: ", useless) 